<a href="https://colab.research.google.com/github/CryptoRobotFr/TrueStrategy/blob/main/dca/DCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance 
!pip install ta

In [18]:
# -- Import --
import pandas as pd
from binance.client import Client
import ta
import matplotlib.pyplot as plt
import numpy as np

# -- Define Binance Client --
client = Client()

# -- You can change the crypto pair ,the start date and the time interval below --
pairName = "ETHUSDT"
startDate = "01 january 2017"
timeInterval = Client.KLINE_INTERVAL_1WEEK

# -- Load all price data from binance API --
klinesT = client.get_historical_klines(pairName, timeInterval, startDate)

# -- Define your dataset --
df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
df['close'] = pd.to_numeric(df['close'])
df['high'] = pd.to_numeric(df['high'])
df['low'] = pd.to_numeric(df['low'])
df['open'] = pd.to_numeric(df['open'])

# -- Set the date to index --
df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']

print("Data loaded 100%")

# -- Uncomment the line below if you want to check your price dataset --
# df

Data loaded 100%


In [19]:
# -- Indicator variable --
# stochWindow = 14

# -- Drop all columns we do not need --
df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)

# -- Indicators, you can edit every value --
# df['STOCH_RSI'] = ta.momentum.stochrsi(close=df['close'], window=stochWindow)
# df['RSI'] = ta.momentum.rsi(close=df['close'], window=stochWindow)
df['LAST_ATH'] = df['close'].cummax()

print("Indicators loaded 100%")

# -- Uncomment the line below if you want to check your dataset with indicators --
df

Indicators loaded 100%


,open,high,low,close,volume,LAST_ATH
timestamp,,,,,,
2017-08-14,301.13,312.18,278.00,299.10,21224.89324000,299.10
2017-08-21,299.10,348.13,144.21,348.13,45233.88589000,348.13
2017-08-28,348.11,394.39,320.08,341.77,33886.41427000,348.13
2017-09-04,339.29,344.55,235.55,293.50,35485.23332000,348.13
2017-09-11,293.51,316.46,192.00,257.55,58259.63439000,348.13
...,...,...,...,...,...,...
2021-10-04,3417.73,3695.00,3270.00,3414.22,3329200.94840000,3950.29
2021-10-11,3414.22,3971.00,3371.00,3846.85,3025204.91330000,3950.29
2021-10-18,3847.00,4375.00,3676.00,4082.33,3218953.34257000,4082.33


In [20]:
dfTest = df[:]
weeklyAmount = 30
takerFee = 0.0007
buyAmount = weeklyAmount
mediumBuy = 0
totalInvest = 0
btcWallet = 0

for index, row in dfTest.iterrows():
    totalInvest += buyAmount
    buyBTC = buyAmount / row['close']
    btcWallet += buyBTC - takerFee * buyBTC
    mediumBuy += 1

resultInDollar = btcWallet * dfTest.iloc[-1]['close']
perfInPct = (resultInDollar - totalInvest)/totalInvest
buyAndHoldPerf = (dfTest.iloc[-1]['close'] - dfTest.iloc[0]['close'])/dfTest.iloc[0]['close']
print('Buy',mediumBuy,'time',weeklyAmount,'$')   
print('Total invest', totalInvest, '$')
print('Final wallet', round(btcWallet,3), 'BTC')
print('Final wallet equivalent', round(resultInDollar,2), '$')
print('Performance',round(perfInPct*100,2), '%')
print('Buy and Hold performance', round(buyAndHoldPerf*100,2), '%')

Buy 221 time 30 $
Total invest 6630 $
Final wallet 22.975 BTC
Final wallet equivalent 102995.79 $
Performance 1453.48 %
Buy and Hold performance 1398.83 %


In [22]:
dfTest = df[:]
weeklyAmount = 30
takerFee = 0.0007
buyAmount = 0
bigBuy = 0
mediumBuy = 0
lowBuy = 0
totalEntry = 0
totalInvest = 0
btcWallet = 0

for index, row in dfTest.iterrows():
    if row['close'] <= 0.5 * row['LAST_ATH']:
        buyAmount = 2 * weeklyAmount
        bigBuy+=1
    elif row['close'] > 0.5 * row['LAST_ATH'] and row['close'] <= 0.8 * row['LAST_ATH']:
        mediumBuy+=1
        buyAmount = 1 * weeklyAmount
    elif row['close'] > 0.8 * row['LAST_ATH']:
        lowBuy+=1
        buyAmount = 0.5 * weeklyAmount
        # buyAmount = 0
    totalInvest += buyAmount
    buyBTC = buyAmount / row['close']
    btcWallet += buyBTC - takerFee * buyBTC
    totalEntry += 1

resultInDollar = btcWallet * dfTest.iloc[-1]['close']
perfInPct = (resultInDollar - totalInvest)/totalInvest
buyAndHoldPerf = (dfTest.iloc[-1]['close'] - dfTest.iloc[0]['close'])/dfTest.iloc[0]['close']
print('Buy',bigBuy,'time',2*weeklyAmount,'$')   
print('Buy',mediumBuy,'time',1*weeklyAmount,'$')   
print('Buy',lowBuy,'time',0.5*weeklyAmount,'$')   
print('Total invest', totalInvest, '$')
print('Final wallet', round(btcWallet,3), 'BTC')
print('Final wallet equivalent', round(resultInDollar,2), '$')
print('Performance',round(perfInPct*100,2), '%')
print('Buy and Hold performance', round(buyAndHoldPerf*100,2), '%')

Buy 142 time 60 $
Buy 27 time 30 $
Buy 52 time 15.0 $
Total invest 10110.0 $
Final wallet 42.11 BTC
Final wallet equivalent 188777.69 $
Performance 1767.24 %
Buy and Hold performance 1398.83 %
